# **Relevancy score**

Authors : F.C, S.B

Date : January 13, 2021

**POC : depuis un data frame, si il y a des mots en commun entre les requêtes et le titre ou entre les requêtes et le résumé, les ajouter dans une nouvelle colonne de DataFrame**

In [ ]:
!pip install deplacy
!python -m spacy download fr_core_news_sm

     |████████████████████████████████| 14.7MB 1.1MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=93829b2127086e36a33dd26b3d4e3e9d08617e2464b23f413755341c27bcfe03
  Stored in directory: /tmp/pip-ephem-wheel-cache-kzyr_6n0/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [ ]:
import pandas as pd
import pkg_resources,imp
imp.reload(pkg_resources)
import spacy
import re
import numpy as np
nlp=spacy.load("fr_core_news_sm")
import nltk
nltk.download('stopwords')
stopword=nltk.corpus.stopwords.words('french')
nltk.download('wordnet')
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
wn=nltk.WordNetLemmatizer()
def tokenize(text):
    tokens = re.split('\W+', text) #W+ means a word character or - can go there
    return tokens
def remove_stopwords(tokenzed_list):
    text=[word for word in tokenzed_list if word not in stopword]
    return text
def lemmatizing(tokenized_text):
    texte=[]
    for elem in tokenized_text:
      texte.append(stemmer.stem(elem))
    return texte
def return_mean_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner la moyenne des vecteurs pour chaque phrase
    return np.mean([(X.vector) for X in doc], axis=0)

In [ ]:
def return_word_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le vecteur lié à chaque token
    return [(X.vector) for X in doc]
def return_token(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le texte de chaque token
    return [X.text for X in doc]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_test = pd.read_csv('/content/drive/My Drive/df_test.csv',delimiter=';')

In [ ]:
df_test['query'][0]='(machine learning AND test) OR (adorer AND du futur AND ntm) OR (adorer AND dog)'
df_test['resume'][0]='adorer le machine learning et du futur'

In [ ]:
df_test['mots'] = ''

In [ ]:
df_new=pd.read_csv('/content/drive/My Drive/df_test.csv',delimiter=';')

In [ ]:
df_new['query'][0]='(machine learning AND test) OR (adorer AND du futur AND ntm) OR (adorer AND dog)'
df_new['resume'][0]='adorer le machine learning et du futur'
df_new['mots'] = ''

In [ ]:
#on a deux data frame, un qui fera les comparaisons un qui gardera les mots d'origine

df_new

,url,query,title,resume,mots
0,https://graylinegroup.com/machine-learning-and...,(machine learning AND test) OR (adorer AND du ...,Machine learning and the new innovation paradigm,adorer le machine learning et du futur,
1,https://graylinegroup.com/machine-learning-and...,machine learning AND innovation,Machine learning and the new innovation paradigm,Machine learning is very innovative. It's...,


In [ ]:
df_test['title'] = [x.lower() for x in df_test.title]

In [ ]:
for i in range (len(df_test)):
  df_test['query'][i] = df_test['query'][i].lower()

In [ ]:
import nltk
nltk.download('stopwords')
import pandas as pd
import pickle
import time
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))
from nltk.stem import LancasterStemmer
import numpy as np
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.stem import SnowballStemmer
def cleandesc(desc):
    sent = desc
    sent = "".join([x.lower() if x.isalpha()  else " " for x in sent])
    Porter=SnowballStemmer('french')
    sent = " ".join([Porter.stem(x) if x.lower() not in stop_words  else "" for x in sent.split()])
    sent = " ".join(sent.split())
    return sent
start_time=time.time()

#On prends la racine des mots pour le data frame df_test pour faire nos tests de comparaison

df_test['title']= [cleandesc(x.title) for x in df_test.itertuples()]
df_test['resume']= [cleandesc(x.resume) for x in df_test.itertuples()]
df_test['query']= [cleandesc(x.query) for x in df_test.itertuples()]
end_time=time.time()
print("total time : {} mn".format((end_time-start_time)/60))

total time : 0.00010286172231038412 mn


In [ ]:
df_test

,url,query,title,resume,mots
0,https://graylinegroup.com/machine-learning-and...,machin learning and test or ador and futur and...,machin learning and the new innov paradigm,ador machin learning futur,
1,https://graylinegroup.com/machine-learning-and...,machin learning and innov,machin learning and the new innov paradigm,machin learning is very innov it,


In [ ]:
#On supprime les parenthèses de la requête et la met en minuscule pour le data frame qui servira à la sortie
for i in range(len(df_new)):
  df_new['query'][i]=df_new['query'][i].lower()
  df_new['query'][i]=df_new['query'][i].replace("(","")
  df_new['query'][i]=df_new['query'][i].replace(")","")

In [ ]:
df_new

,url,query,title,resume,mots
0,https://graylinegroup.com/machine-learning-and...,machine learning and test or adorer and du fut...,Machine learning and the new innovation paradigm,adorer le machine learning et du futur,
1,https://graylinegroup.com/machine-learning-and...,machine learning and innovation,Machine learning and the new innovation paradigm,Machine learning is very innovative. It's...,


In [ ]:
def clear_out(df,df_n) :
  for j in range (len(df)): # pour chaque ligne du df,
    separator_or = list(df['query'][j].split(' or ')) #on stocke tous les couples de la requete dans une liste separator_or
    sep_or= list(df_n['query'][j].split(' or '))
    list_test=[] #liste pour stocker les mots en commun pour chaque ligne
    for k in range (len(separator_or)): #pour chacun de ces couples,s
      separator_and = list(separator_or[k].split(' and ')) #on stocke tous les mots du couple de la requete dans une liste separator_and
      sep_and = list(sep_or[k].split(' and '))
      for i in range (len(separator_and)): #pour tous les mots de la requete,
        if (df['title'][j].find(separator_and[i]) != -1) : #on le cherche dans le titre, et si c'est présent...
          list_test.append(sep_and[i])
          print(sep_and[i])
        if (df['resume'][j].find(separator_and[i]) != -1) : #ensuite, on le cherche dans le resumé, et si c'est présent...
          list_test.append(sep_and[i]) #... on le met dans une liste
          list_test = list(set(list_test))
          print(sep_and[i])
      str = ' '.join(list_test) #...
    df_n['mots'][j]=list_test
  return(df_n)


In [ ]:
dt = clear_out(df_test,df_new)
dt

machine learning
machine learning
adorer
du futur
adorer
machine learning
machine learning
innovation
innovation


,url,query,title,resume,mots
0,https://graylinegroup.com/machine-learning-and...,machine learning and test or adorer and du fut...,Machine learning and the new innovation paradigm,adorer le machine learning et du futur,"[machine learning, du futur, adorer]"
1,https://graylinegroup.com/machine-learning-and...,machine learning and innovation,Machine learning and the new innovation paradigm,Machine learning is very innovative. It's...,"[machine learning, innovation]"
